In [ ]:
import os
import json
import collections
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
inverted_index_folder = "/4  курс/Информационный поиск/task3"
tokens_folder = "/4  курс/Информационный поиск/task2"
write_folder = "/4  курс/Информационный поиск/task4"

In [ ]:
token_inverted_index_count = {}
tokens_dictionary = {}
lemmas_dictionary = {}

In [ ]:
files_count = len(tokens_dictionary)

In [ ]:
# сохраняем все токены из всех файлов в словарь tokens_dictionary 
# {'0': ['token1', 'token2'], ...}
for dirpath, _, filenames in os.walk('/content/drive/My Drive/'+ tokens_folder):
  for filename in filenames:
    if 'tokens-' in filename: 
      f = open('/content/drive/My Drive/'+ tokens_folder +'/' + filename, encoding='utf-8')
      data = f.read()
      name = filename.replace("tokens-", "").replace(".txt", "")
      data_list = data.splitlines()
      tokens_dictionary[name] = data_list

In [ ]:
# значения из 3 задания сохраняем в словарь token_inverted_index_count для удобной работы
# {'token1': 10, ...}
with open('/content/drive/My Drive/'+ inverted_index_folder + '/inverted_index_2.txt', 'r') as f:
    data = f.readlines()
    for elem in data:
        elem = elem.replace("\'", "\"")
        string_data = json.loads(elem)
        token_inverted_index_count[string_data['word']] = string_data['count']

In [ ]:
for key, token_list in tokens_dictionary.items():
    tf_idf_dictionary = {}

    # считаем tf - отношение количества вхождений токена в файл ко всем токенам в файле
    token_tf = collections.Counter(token_list)
    token_list_len = len(token_list)
    for i in token_tf:
        token_tf[i] = token_tf[i]/float(token_list_len)

    # считаем idf - отношение общего количества файлов к количеству файлов, содержащих токен
    # считаем tf-idf - tf * idf
    for word in token_tf:
        idf = math.log10(files_count/token_inverted_index_count[word])

        tf_idf_dictionary[word] = {
            'idf': idf,
            'tf-idf': token_tf[word] * idf
        }

    # запишем в файл
    with open('/content/drive/My Drive/' + write_folder + '/tokens-' + key + '.txt', 'w') as f:
        for token, metric in tf_idf_dictionary.items():
            result_string = token + " " + str(metric['idf']) + " " + str(metric['tf-idf'])
            f.write("%s\n" % result_string)
        print(key)

In [ ]:
# считаем для лемм

for dirpath, _, filenames in os.walk('/content/drive/My Drive/'+ tokens_folder):
  for filename in filenames:
    if 'lemmas-' in filename: 
      f = open('/content/drive/My Drive/'+ tokens_folder +'/' + filename, encoding='utf-8')
      data = f.read()
      name = filename.replace("lemmas-", "").replace(".txt", "")
      data_list = data.splitlines()
      lemmas_dictionary[name] = data_list

In [ ]:
for key, lemmas_list in lemmas_dictionary.items():
  lemma_tf = {}
  token_tf = collections.Counter(tokens_dictionary[key])
  token_list_len = len(tokens_dictionary[key])

  for v in lemmas_list:
    v = v.strip().split(':')
    lemma = v[0].strip()
    tokens = v[1].strip().split(" ")
    # print(lemma, ":", tokens)
    # print(tokens)
    token_count = 0
    idf = float(0)
    for word in tokens:
      token_count += token_tf[word]
      idf += math.log10(files_count/token_inverted_index_count[word])

    tf = token_count / float(token_list_len)

    lemma_tf[lemma] = [idf, tf * idf]

  with open('/content/drive/My Drive/' + write_folder + '/lemmas-' + key + '.txt', 'w') as f:
    for lemma, metrics in lemma_tf.items():
      result_string = lemma + " " + str(metrics[0]) + " " + str(metrics[1])
      f.write("%s\n" % result_string)
  print(key)